# Mamba vs. Transformer-based RALMs

The purpose of this project is to compare the performance of Retrieval Augmented Language Models (RALMs) based on the newly released Mamba architecture to those based on the more prevalent Transformer architecture. We will compare the [Mamba-Chat](https://huggingface.co/havenhq/mamba-chat) model to the [Dolly](https://huggingface.co/databricks/dolly-v2-3b) model. Both these models are approximately 2.8B parameters.

## Part 0: Imports and Environment Setup

First, to facilitate the use of this notebook in colab, load the rest of the repo. ONLY RUN THIS CODE IN COLAB.

In [ ]:
!wget -q https://raw.githubusercontent.com/tsunrise/colab-github/main/colab_github.py
import colab_github
colab_github.github_auth(persistent_key=True)

In [ ]:
#after updating ssh key on github account if necessary, run below
!git clone git@github.com:abarton51/CS_4650_Project.git
!mv CS_4650_Project/* .

Next, install the required dependencies.

In [ ]:
!pip install datasets
!pip install transformers
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm

In [ ]:
import sys
sys.path.append("src/")
from datasets import load_dataset
from mamba_ralm import MambaRALM

## Part 1: Load Dataset

Here we load in the relevant benchmark dataset.

In [ ]:
dataset = load_dataset("natural_questions")

# Part 2: Construct Vector Database

Now let us construct a Vector Database

# Part 3: Initialize Model

Initialize the model. Select the desired model here.

In [ ]:
model = MambaRALM()